In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

In [ ]:
import os
mcpurl = "http://13.201.22.227:8000/mcp"

In [11]:

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

async with streamablehttp_client("http://13.232.211.169:8000/mcp/") as (read, write, _):
    async with ClientSession(read, write) as client:
        # Initialize the connection
        await client.initialize()
        tools = await load_mcp_tools(client)
        print(tools)
        model_with_tools = model.bind_tools(tools)
        tools_node = ToolNode(tools=tools)
        app = create_graph(model_with_tools, tools_node)
        print(await app.ainvoke({"messages": [{"role": "user", "content": "what's the weather in sf?"}]}))


[StructuredTool(name='generate_salesforce_oauth_url', description='\n    Generates a Salesforce OAuth2 authorization URL using values from environment variables\n    and a provided profile_id.\n    \n    Env Variables:\n    - SALESFORCE_DOMAIN\n    - SALESFORCE_CLIENT_ID\n    - SALESFORCE_REDIRECT_URI\n    \n    Args:\n        profile_id (str): The user profile ID to be passed in the state parameter.\n    \n    Returns:\n        str: Complete Salesforce OAuth2 URL.\n    ', args_schema={'properties': {'profile_id': {'title': 'Profile Id', 'type': 'string'}}, 'required': ['profile_id'], 'title': 'generate_salesforce_oauth_urlArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7fc98d9e2e60>), StructuredTool(name='execute_salesforce_soql', description="\n    Executes a SOQL query against Salesforce using credentials fetched from DynamoDB.\n\n    Args:\n        soql_query (str): The SOQL query 

In [3]:
from typing import Literal

from langgraph.graph import StateGraph, MessagesState, START, END

def create_graph(model_with_tools, tool_node):
    def should_continue(state: MessagesState):
        messages = state["messages"]
        last_message = messages[-1]
        if last_message.tool_calls:
            return "tools"
        return END


    async def call_model(state: MessagesState):
        messages = state["messages"]
        response = await model_with_tools.ainvoke(messages)
        return {"messages": [response]}


    workflow = StateGraph(MessagesState)

    # Define the two nodes we will cycle between
    workflow.add_node("agent", call_model)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", should_continue, ["tools", END])
    workflow.add_edge("tools", "agent")

    app = workflow.compile()
    return app